# Dominant colors

To drive the Hue lights we need to determine the color they should be set to.
The simplest approach is to calculate the mean of the RGB components.  However,
this tends to result in a washed out look.  Colors average to brown or gray.

Other algorithms using clustering.  K-means is the most promising of these algorithms.
Unfortunately k-means is computationally expensive and performance suffers when performed
on every video frame.

This notebook experiments with different implementations.

## Obtain a test image

In [5]:
from PIL import Image
import numpy as np

image = Image.open("examples/colors.jpg")

print(image.mode)

image_data = np.asarray(image, dtype=np.uint8)

print(image_data.shape)

RGB
(1200, 1920, 3)


In [32]:
import dominantcolors
import time
import pandas as pd
import cv2
from IPython.display import HTML
from sklearn.cluster import KMeans
from sklearn.utils import shuffle


def algo_mean(data) -> tuple[int, int, int]:
    r = int(np.mean(data[:,:,0]))
    g = int(np.mean(data[:,:,1])) 
    b = int(np.mean(data[:,:,2]))
    
    return (r, g, b)

def algo_dominantcolor_lib(data, num_color) -> tuple[int, int, int]:
    dominant_colors = dominantcolors.find_dominant_colors(data, num_color)
    (r,g,b) = dominant_colors[0]
    return (r, g, b)

def algo_sklearn(data, num_color):
    kmeans = KMeans(n_clusters=num_color, init='k-means++').fit(data.reshape(-1,3))
    centroids = kmeans.cluster_centers_.astype("uint8")
    return tuple(centroids[0])
    
def algo_sklearn_sub(data, num_color):
    image_array_sample = shuffle(data.reshape(-1,3), random_state=0, n_samples=1_000)
    kmeans = KMeans(n_clusters=num_color, random_state=0).fit(image_array_sample)
    centroids = kmeans.cluster_centers_.astype("uint8")
    return tuple(centroids[0])
    

results = []
images = [
    image_data,
    cv2.resize(image_data, (640, 320), interpolation = cv2.INTER_AREA),
    cv2.resize(image_data, (320, 160), interpolation = cv2.INTER_AREA),
    cv2.resize(image_data, (160, 80), interpolation = cv2.INTER_AREA)
]
alogorithms = ["mean", "dc_lib", "sklearn", "sklearn_sub"]
num_colors = [2,3]
num_iterations = 5

for algo in alogorithms:
    for im in images:
        for num_color in num_colors:
            times = []
            result = None
            for _ in range(num_iterations):
                start = time.time()
                
                if algo == "mean":
                    result = algo_mean(im)
                elif algo == "dc_lib":
                    result = algo_dominantcolor_lib(im, num_color)
                elif algo == "sklearn":
                    result = algo_sklearn(im, num_color)
                elif algo == "sklearn_sub":
                    result = algo_sklearn_sub(im, num_color)
                else:
                    raise Exception("Invalid algorithm")
                    
                times.append(time.time()-start)
                
            # Only take the last result,  Assume that is is representative.
            results.append((result, algo, im.shape, num_color, np.mean(times), np.std(times)))
    
results_df = pd.DataFrame(results, columns=["Color", "Algorithm", "Size", "#Color", "Mean Time", "Standard Deviation"])

def rgb_to_color_swatch(rgb):
    r, g, b = rgb
    return f'<div style="background-color:rgb({r},{g},{b}); width:50px; height:20px; border:1px solid black;"></div>'

# Convert the Color column to HTML color swatches
results_df['Color'] = results_df['Color'].apply(rgb_to_color_swatch)

HTML(results_df.to_html(escape=False))

    


,Color,Algorithm,Size,#Color,Mean Time,Standard Deviation
0,,mean,"(1200, 1920, 3)",2,0.004196,1.755270e-04
1,,mean,"(1200, 1920, 3)",3,0.004295,8.213517e-05
2,,mean,"(320, 640, 3)",2,0.000383,3.518380e-05
3,,mean,"(320, 640, 3)",3,0.000460,1.780307e-04
4,,mean,"(160, 320, 3)",2,0.000123,2.412598e-05
5,,mean,"(160, 320, 3)",3,0.000126,2.937735e-05
6,,mean,"(80, 160, 3)",2,0.000035,2.937874e-06
7,,mean,"(80, 160, 3)",3,0.000034,8.449576e-07
8,,dc_lib,"(1200, 1920, 3)",2,0.265298,4.968324e-03
9,,dc_lib,"(1200, 1920, 3)",3,0.361425,5.228648e-03
